# Impact of Bedmachine on ice flow modelling thourgh version 1-5
### Group Project International Summer School in Glaciology, McCarthy, June 7-17 2024

## **Is Bedmachine getting better?**

NASA’s Operation IceBridge (2009-2019) changed our understanding of the subglacial topography beneath the Greenland Ice Sheet. Radar-derived Ice thickness along flightlines have been gridded using kriging and algorithms based on mass conservation, resulting in the now widely-used ”BedMachine Greenland”. A manuscript in 2016 showed that BedMachine Greenland significantly improves the ability of ice flow models to reproduce the flow patterns of Greenland’s outlet glaciers. The study was done using BedMachine version 1, however, the release version is version 5. In this project we will investigate the changes in ice thickness in BedMachine throughout the different versions, and how it affects an ice flow model’s fidelity in reproducing outlet glacier flow. 

